In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Installer les dépendances nécessaires
!pip install tensorflow-text transformers

# Charger les données
train_data = pd.read_csv('/kaggle/input/training-data/training_data.csv')

# Séparation des caractéristiques et des labels
X_train, X_val, y_train, y_val = train_test_split(train_data['sentence'], train_data['difficulty'], test_size=0.2, random_state=42)


In [4]:
# Encodage des labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Initialiser le tokenizer DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

# Fonction de tokenisation
def tokenize(sentences, max_length=128):
    return tokenizer(sentences, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

train_encodings = tokenize(X_train.tolist(), max_length=128)
val_encodings = tokenize(X_val.tolist(), max_length=128)


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [5]:
# Charger le modèle DistilBERT (version TensorFlow)
bert = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [6]:
# Construction du modèle
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(128,), dtype=tf.int32, name="attention_masks")

bert_outputs = bert(input_ids, attention_mask=attention_masks).last_hidden_state
output = tf.keras.layers.GlobalAveragePooling1D()(bert_outputs)
output = Dense(128, activation='relu')(output)
output = Dropout(0.5)(output)
output = Dense(len(label_encoder.classes_), activation='softmax')(output)

model = Model(inputs=[input_ids, attention_masks], outputs=output)
model.compile(optimizer=Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraînement du modèle
history = model.fit(
    {'input_ids': train_encodings['input_ids'], 'attention_masks': train_encodings['attention_mask']},
    y_train_encoded,
    validation_data=(
        {'input_ids': val_encodings['input_ids'], 'attention_masks': val_encodings['attention_mask']},
        y_val_encoded
    ),
    epochs=3,
    batch_size=16
)

Epoch 1/3
240/240 [==============================] - 67s 194ms/step - loss: 1.3639 - accuracy: 0.4112 - val_loss: 1.1242 - val_accuracy: 0.5167
Epoch 2/3
240/240 [==============================] - 37s 154ms/step - loss: 1.0812 - accuracy: 0.5349 - val_loss: 1.1194 - val_accuracy: 0.5146
Epoch 3/3
170/240 [====================>.........] - ETA: 9s - loss: 0.7753 - accuracy: 0.6890